In [15]:
import os
import cv2
import numpy as np

import torch
from fastai.vision.all import load_learner, PILImage
from fastxtend.vision.all import adam

In [10]:
model_path_fastai = os.path.join(os.path.curdir, r"model.pkl")
model_path_torch = os.path.join(os.path.curdir, r"model.pt")

In [11]:
def round_pred(pred, threshold=0):
    if pred < 0.4 and pred > threshold:
        return 0.4
    elif pred < threshold:
        return -1
    else:
        return pred

def predict(img_path, model_path=model_path_fastai):
    """predicts steer and throttle given an image path

    Parameters:
    ---
    img_path: str
        path to image

    Returns:
    ---
    steer: float
        predicted steer value
    throttle: float
        predicted throttle value
    """
    # learner = Learner().load(model_path)
    learner = load_learner(model_path)
    img = PILImage.create(img_path)
    steer, throttle = learner.predict(img)
    throttle = round_pred(throttle, -0.5)
    return steer, throttle

In [12]:
print(predict(r"curved_109.png"))

NotImplementedError: cannot instantiate 'PosixPath' on your system

In [13]:
def predict_torch(img_path, model_path=model_path_torch):
    """predicts steer and throttle given an image path

    Parameters:
    ---
    img_path: str
        path to image

    Returns:
    ---
    steer: float
        predicted steer value
    throttle: float
        predicted throttle value
    """
    model = torch.load(model_path)   
    model = model.double() 
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = np.expand_dims(img, axis=0)
    img = img.transpose(0, 3, 1, 2)
    img = img / 255
    img = torch.from_numpy(img).double()

    steer, throttle = model.forward(img).tolist()[0]
    throttle = round_pred(throttle, -0.5)
    return steer, throttle

In [16]:
print(predict_torch(r"curved_109.png"))

(-0.05683895938811283, 0.5528247570058203)
